# Quantum state preparation without coherent arithmetic

This notebook implements a quantum state preparation circuit based on _Quantum State Preparation Without Coherent Arithmetic_ ([arXiv:2210.14892](https://arxiv.org/pdf/2210.14892)), as part of “Implementation Challenge” by Classiq. 
 Specifically, we will construct a Gaussian state preparation circuit from [Classiq’s MIT iQuHack 2025 challenge](https://github.com/iQuHACK/2025-Classiq/blob/main/classiq_iQuHack_2025_final.ipynb) using the approach outlined in this paper.

$$|{x_0}\rangle_N = |0\rangle_N \rightarrow \sum_x {G(x)} |{x}\rangle_N$$

## Algorithm Overview

The algorithm leverages Quantum Eigenvalue Transformation (QET) technique to encode a function into a quantum state. 
The key idea is to approximate the desired function using polynomial approximations and implement these polynomials as quantum circuits.

#### General workflow
- The algorithm starts by defining a function $f(x)$ (in this case, a Gaussian function) to be encoded into the quantum state.
- It constructs a block-encoding circuit ($U_{\text{sin}}$) that initializes the input state.
- Using QET, the function $f(x)$ is applied to the input state via a circuit ($U_f$) that encodes the polynomial approximation of $f(x)$.
- Finally, amplitude amplification ($U_{\text{amp}}$) is applied to increase the probability of measuring the desired state.

#### Benefits
- Scalability: The algorithm avoids the need for coherent arithmetic, making it more efficient for amplitude encoding. It requires only a fixed number of ancillary qubits (at most 4) regardless of the function's complexity.
- Flexibility: It supports encoding various functions, such as polynomial or Fourier series approximations, and is applicable to a wide range of continuous functions.
- Resource Efficiency: By leveraging QET, the algorithm minimizes the number of non-Clifford gates and logical qubits while avoiding the use of amplitude oracles.

#### Complexity
- The complexity of the algorithm depends on the degree of the polynomial approximation used for `f(x)`. Higher-degree polynomials provide better approximations but require more quantum gates.
- Specifically, the non-Clifford gate complexity of the QET-based approach is approximately:
            $$ O\left(\frac{n \cdot d}{F}\right), $$
where:
    - $n$: Number of qubits representing the input state.
    - $d$: Degree of the polynomial approximation.
    - $F$: L2-norm filling-fraction of the polynomial approximation.

This complexity indicates that the number of non-Clifford gates scales linearly with the number of qubits $n$ and the polynomial degree $d$.
For detailed analysis, please refer to the paper.

## Implementation

To implement amplitude encoding without coherent arithmetic, we construct the following three components, as described in Figure 1 of the paper:
- $U_{\text{sin}}$: Block encoding of the initial values.
- $U_f$: Function application via Quantum Eigenvalue Transformation (QET).
- $U_{\text{amp}}$: Amplitude amplification to increase the probability of the target state.

For each component, we follow Section 2 of the paper. In constructing $U_f$, we determine the QET rotation angles using [pyqsp](https://github.com/ichuang/pyqsp) or other techniques. For evaluation, we verify correctness via state-vector simulation for small cases and perform resource estimation.

For Gaussian state preparation, we set `f(x)` to be the Gaussian function described in the[ MIT iQuHack 2025 challenge notebook](https://github.com/iQuHACK/2025-Classiq/blob/main/classiq_iQuHack_2025_final.ipynb).

In [ ]:
import classiq
from classiq import *

print(classiq.__version__)

0.84.0


We simulate the circuit and check if the Gaussian state is prepared correctly.

In [ ]:
@qfunc
def main() -> None:
    qn = QNum()
    allocate(5, qn)
    hadamard_transform(qn)
    q = QBit()
    q |= qn == 0

qmod = create_model(
    main,
    execution_preferences=ExecutionPreferences(
        num_shots=1,
        backend_preferences=ClassiqBackendPreferences(
            backend_name="simulator_statevector"
        ),
    ),
    preferences=Preferences(timeout_seconds=1800),
)
write_qmod(qmod, "stateprep_guassian_using_qsvt")
qprog = synthesize(qmod)
show(qprog)
result = execute(qprog).result_value()

Quantum program link: https://platform.classiq.io/circuit/2zDurVuhGLLyw0xv4FYOACpvsnz
